In [2]:
import os 
from pdf2image import convert_from_path
import cv2
import pytesseract
from concurrent.futures import ProcessPoolExecutor, as_completed

ModuleNotFoundError: No module named 'pdf2image'

In [5]:
def extraer_texto_pdf(pdf_path):
    #Extrae el texto 
    try:
        pages = convert_from_path(pdf_path, dpi = 400)
        texts = []

        for i, page in enumerate(pages):
            #guarda imagen como temporal
            temp_img_path = pdf_path.replace('.pdf', f'_page_{i}.jpg')
            page.save(temp_img_path, 'JPEG')

            #carga imagen y procesa con cv2
            img = cv2.imread(temp_img_path)
            if img is None:
                print(f"No se puede leer la imagen {temp_img_path}")
                continue

             # Verificar el número de canales
            if len(img.shape) == 2:
                # Ya está en escala de grises
                gray = img
            elif img.shape[2] == 4:
                # Imagen con canal alfa (BGRA)
                gray = cv2.cvtColor(img, cv2.COLOR_BGRA2GRAY)
            elif img.shape[2] == 3:
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            else:
                gray = img  # Si no coincide, se deja tal cual

            thresh = cv2.threshold(gray,0,255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

            #extrae con tesseract
            custom_config = r'--oem 3 --psm 6'
            text = pytesseract.image_to_string(thresh, config=custom_config, lang='eng')
            texts.append(text)

            #eliminar imagen temporal
            os.remove(temp_img_path)

        return "\n".join(texts)
    except Exception as e:
        return f"Error procesando {pdf_path} : {e}"

In [6]:
def procesar_pdfs(root_dir):
    pdf_texts = {}
    for dirpath, _, filenames in os.walk(root_dir):
        for file in filenames:
            if file.lower().endswith('.pdf'):
                pdf_path = os.path.join(dirpath,file)
                print(f"Procesando {pdf_path}")
                text = extraer_texto_pdf(pdf_path)
                pdf_texts[pdf_path] = text
    return pdf_texts

In [10]:
if __name__ == '__main__':
    #se define carpeta raiz donde se encuentran todos los pdfs
    root_directory = '/home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/Mitsubishi' 
    extracted_data = procesar_pdfs(root_directory)

    for pdf_path, text in extracted_data.items():
        txt_path = pdf_path.replace('.pdf', '.txt')
        with open(txt_path, 'w', encoding='utf-8') as f:
            f.write(text)
        print(f"Guardando el texto extraido en {txt_path}")

Procesando /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/Mitsubishi/fichatecnica-outlander-sport.pdf
Procesando /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/Mitsubishi/specs-new-montero-sport.pdf
Procesando /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/Mitsubishi/fichatecnica-newl200.pdf
Procesando /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/Mitsubishi/specs-new-outlander.pdf
Procesando /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/Mitsubishi/specs-new-asx.pdf
Procesando /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/Mitsubishi/specs-xpander.pdf
Procesando /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/Mitsubishi/specs-eclipse-cross.pdf
Procesando /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/Mitsubishi/specs-xpander-cross.pdf
Guardando el texto extraido en /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/